Let $G \subset S_n$ be a permutation group. There is a natural action of $G$ on $X^{(d)}$, the set of subsets of $X = \{1, \dots, n\}$ of size $d$. Let $\phi_d$ be the character of the corresponding permutation representation.

The character $\phi_d(g)$ of a permutation $g$ acting on the set of $d$-element subsets is the number of $d$-element subsets that are fixed by $g$. A subset $S$ is fixed if $g(S) = S$, which means that $S$ must be a union of some of the disjoint cycles of $g$. To calculate this, we need the cycle structure of $g$. Let $c_k$ be the number of cycles of length $k$ in the disjoint cycle decomposition of $g$.
For instance, for $\phi_2(g)$, we need to form a fixed subset of size $2$. This can be done in two ways:
*   There are $c_2$ ways of choosing one $2$-cycle.
*   There are $\binom{c_1}{2}$ ways of choosing two $1$-cycles.
Therefore, $\phi_2(g) = c_2 + \binom{c_1}{2}$.
For $\phi_3(g)$, we need to form a fixed subset of size $3$. There are three way to do this:
*   There are c_3 ways to choose one $3$-cycle
*   There are $c_2c_1$ ways to choose a $2$-cycle and a $1$-cycle.
*   There are $\binom{c_1}{3}$ to choose three $1$-cycles.

Therefore, $\phi_3(g) = c_3 + c_1c_2 + \binom{c_1}{3}$.

Any character $\phi$ can be written as a sum of the irreducible characters $\phi = \sum_i a_1\chi_i$. The multiplicity of $\chi_i$ in $\phi$ is the coefficient $a_i$ which is a non-negative integer given by the character inner product:
\begin{equation}
    a_i = \langle \phi, \chi_i \rangle = \frac{1}{|G|} \sum_{g \in G} \phi(g) \chi_i(g^{-1}).
\end{equation}
Since characters are constant on conjugacy classes and $\chi_i(g^{-1}) = \overline{\chi_i(g)}$, we can use the more efficient formula\
\begin{equation}
    a_i = \frac{1}{|G|} \sum_{j=1}^r |C_j| \phi(g_j) \overline{\chi_i(g_j)},
\end{equation}
where the sum is over the $r$ conjugacy classes $C_j$ with representatives $g_j$.

In [5]:
import numpy as np
import collections
import random
import re
import math

class Permutation:
    def __init__(self, mapping):
        self.mapping = tuple(mapping)
        self.size = len(mapping)

    def __mul__(self, other):
        return Permutation([self.mapping[other.mapping[i]] for i in range(self.size)])

    def inverse(self):
        inv_mapping = [0] * self.size
        for i, val in enumerate(self.mapping): inv_mapping[val] = i
        return Permutation(inv_mapping)

    def __eq__(self, other): return self.mapping == other.mapping
    def __hash__(self): return hash(self.mapping)
    def __lt__(self, other): return self.mapping < other.mapping

    def __repr__(self):
        p, n, seen, cycles = list(self.mapping), self.size, [False]*n, []
        for i in range(n):
            if not seen[i]:
                cycle, j = [], i
                while not seen[j]:
                    seen[j] = True
                    cycle.append(j)
                    j = p[j]
                if len(cycle) > 1:
                    cycles.append(tuple(c+1 for c in cycle))
        return "".join(map(str, cycles)) if cycles else "()"

def parse_cycle_notation(cycle_str, n):
    mapping = list(range(n))
    if not cycle_str or cycle_str == "()": return Permutation(mapping)
    for m in re.finditer(r'\(([\d,]+)\)', cycle_str):
        c = [int(x)-1 for x in m.group(1).split(',')]
        if len(c) > 1:
            v, i = mapping[c[-1]], len(c)-1
            while i > 0: mapping[c[i]], i = mapping[c[i-1]], i-1
            mapping[c[0]] = v
    return Permutation(mapping)

def generate_group(generators):
    if not generators: return set()
    identity = Permutation(tuple(range(generators[0].size)))
    group, worklist = {identity}, collections.deque([identity])
    while worklist:
        curr = worklist.popleft()
        for gen in generators:
            new = curr * gen
            if new not in group:
                group.add(new)
                worklist.append(new)
    return group

def find_conjugacy_classes(group):
    if not group: return []
    classes, processed = [], set()
    for g in sorted(list(group)):
        if g not in processed:
            cls = {h * g * h.inverse() for h in group}
            classes.append(sorted(list(cls)))
            processed.update(cls)
    classes.sort(key=lambda c: (len(c), c[0]))
    return [set(c) for c in classes]

def compute_nu_ijk(classes):
    r = len(classes)
    nu = np.zeros((r, r, r), dtype=int)
    elem_to_class = {g: k for k, c in enumerate(classes) for g in c}
    for i in range(r):
        for j in range(r):
            counts = collections.defaultdict(int)
            for g1 in classes[i]:
                for g2 in classes[j]:
                    counts[elem_to_class[g1 * g2]] += 1
            for k, count in counts.items():
                nu[i, j, k] = count // len(classes[k])
    return nu

def compute_character_table(generators):
    group = generate_group(generators)
    order, classes = len(group), find_conjugacy_classes(group)
    r, class_sizes = len(classes), [len(c) for c in classes]
    nu = compute_nu_ijk(classes)
    matrices_N = [nu[i, :, :] for i in range(r)]
    random.seed(42)
    coeffs = [random.randint(-5, 5) for _ in range(r)]
    M = np.sum([c * N for c, N in zip(coeffs, matrices_N)], axis=0)
    _, eigenvectors = np.linalg.eig(M)
    char_table = []
    for i in range(r):
        v = eigenvectors[:, i]
        psi = np.array([v[j] / class_sizes[j] for j in range(r)])
        psi_e = psi[0]
        norm_sum = np.sum([size * abs(val)**2 for size, val in zip(class_sizes, psi)])
        dim_squared = (order * abs(psi_e)**2) / norm_sum
        dim = int(round(np.sqrt(dim_squared).real))
        scaling_factor = dim / psi_e
        chi = psi * scaling_factor
        char_table.append(chi)
    char_table.sort(key=lambda c: (int(round(c[0].real)), -np.sum(c.real)))
    return np.array(char_table), classes, order, generators[0].size

In [6]:
def get_cycle_structure(perm, n):
    '''
    Calculates the cycle structure of a permutation.
    '''
    counts = collections.defaultdict(int)
    seen = [False] * n
    for i in range(n):
        if not seen[i]:
            cycle_len, j = 0, i
            while not seen[j]:
                seen[j], j = True, perm.mapping[j]
                cycle_len += 1
            counts[cycle_len] += 1
    return counts

def calculate_phi_character(d, classes, n):
    '''
    Calculates the permutation character phi_d.
    '''
    phi = np.zeros(len(classes))
    for i, cls in enumerate(classes):
        rep = next(iter(cls)) # Get a representative
        cycles = get_cycle_structure(rep, n)
        c1 = cycles.get(1, 0)
        c2 = cycles.get(2, 0)
        c3 = cycles.get(3, 0)

        if d == 2:
            phi[i] = c2 + math.comb(c1, 2)
        elif d == 3:
            phi[i] = c3 + c1 * c2 + math.comb(c1, 3)
    return phi

def decompose_character(phi, char_table, classes, order):
    '''
    Decomposes phi into a sum of irreducibles using inner products.
    '''
    multiplicities = []
    class_sizes = np.array([len(c) for c in classes])

    for chi in char_table:
        inner_product = (1 / order) * np.sum(class_sizes * phi * np.conj(chi))
        # The result must be a non-negative integer. Round to correct for float errors.
        multiplicities.append(int(round(inner_product.real)))

    return multiplicities

def print_decomposition(d, multiplicities, char_table):
    '''
    Formats the decomposition result as a string.
    '''
    terms = []
    for i, mult in enumerate(multiplicities):
        if mult > 0:
            dim = int(round(char_table[i][0].real))
            term = f"χ_{{{i+1}}}"
            if mult > 1:
                term = f"{mult}{term}"
            terms.append(term)
    print(f"φ_{d} = " + " + ".join(terms))

In [7]:
GROUPS = {
    "S3": {"n": 3, "gens": ["(1,2)", "(1,2,3)"]},
    "A4": {"n": 4, "gens": ["(1,2,3)", "(2,3,4)"]},
    "S4": {"n": 4, "gens": ["(1,2)", "(1,2,3,4)"]},
    "P1": {"n": 8, "gens": ["(1,2,3,4)(5,6,7,8)", "(1,5,3,7)(2,8,4,6)"]}, # Q8
    "P2": {"n": 8, "gens": ["(1,2,3,4)(5,6,7,8)", "(1,8)(2,7)(3,6)(4,5)"]}, # D8
    "G1": {"n": 13,"gens": ["(1,2,3)(4,5,6)(7,8,9)(10,11,12)", "(2,9)(4,11)(5,8)(7,13)"]}, # PSL(2, 13)
    "G2": {"n": 14,"gens": ["(1,2,3,4)(5,6,7,8)(9,10,11,12)(13,14)", "(1,3,14,5,11,7)(2,4,6,10,8,13)"]}, # PSL(2, 17)
    "G3": {"n": 8, "gens": ["(1,2,3,4,5,6,7)", "(1,4)(2,3)(5,8)(6,7)"]}, # C7 ⋊ C8
    #"G4": {"n": 10,"gens": ["(1,2,3,4)(5,6,7,8)", "(3,9,8,10)(4,6,11,7)"]} # M11
}

for name, data in GROUPS.items():
    print(f"\nDecompositions for {name}")
    gens = [parse_cycle_notation(g, data["n"]) for g in data["gens"]]
    table, classes, order, n = compute_character_table(gens)

    # Decompose phi_2
    phi_2 = calculate_phi_character(2, classes, n)
    mults_2 = decompose_character(phi_2, table, classes, order)
    print_decomposition(2, mults_2, table)

    # Decompose phi_3
    phi_3 = calculate_phi_character(3, classes, n)
    mults_3 = decompose_character(phi_3, table, classes, order)
    print_decomposition(3, mults_3, table)


Decompositions for S3
φ_2 = χ_{1} + χ_{3}
φ_3 = χ_{1}

Decompositions for A4
φ_2 = χ_{1} + χ_{2} + χ_{3} + χ_{4}
φ_3 = χ_{1} + χ_{4}

Decompositions for S4
φ_2 = χ_{1} + χ_{3} + χ_{4}
φ_3 = χ_{1} + χ_{3} + χ_{4}

Decompositions for P1
φ_2 = 4χ_{1} + 4χ_{2} + 4χ_{3} + 2χ_{4} + 6χ_{5}
φ_3 = 7χ_{1} + 7χ_{2} + 7χ_{3} + 7χ_{4} + 14χ_{5}

Decompositions for P2
φ_2 = 6χ_{1} + 4χ_{2} + 2χ_{3} + 2χ_{4} + 6χ_{5}
φ_3 = 7χ_{1} + 7χ_{2} + 7χ_{3} + 7χ_{4} + 14χ_{5}

Decompositions for G1
φ_2 = χ_{1} + 2χ_{2} + χ_{8} + χ_{11}
φ_3 = 2χ_{1} + 4χ_{2} + χ_{4} + χ_{5} + χ_{6} + χ_{7} + 2χ_{8} + 3χ_{11} + χ_{12}

Decompositions for G2
φ_2 = 2χ_{1} + χ_{2} + 2χ_{5} + χ_{7} + χ_{8} + χ_{9} + 4χ_{10}
φ_3 = 3χ_{1} + 3χ_{2} + χ_{3} + χ_{4} + 4χ_{5} + 6χ_{7} + 6χ_{8} + 6χ_{9} + 11χ_{10}

Decompositions for G3
φ_2 = χ_{1} + χ_{4} + χ_{7} + χ_{9}
φ_3 = χ_{1} + χ_{4} + 2χ_{7} + χ_{9} + χ_{11}


A group $G$ is simple if it has no non-trivial proper normal subgroups. A group is simple if and only if the kernel (a normal subgroup) of every non-trivial irreducible character is the trivial subgroup.

*  The first column of the table gives the dimension $d_i = \chi_i(e)$ for each irreducible character $\chi_i$.
*   For every row except the first one (the trivial character), check if the value in any other column is equal to the dimension $d_i$..
    *   If the only time this happens is in the first column, then $\ker(\chi_i)$ is trivial. If this is true for all non-trivial characters, then the group is simple.
    *   If there is even one non-trivial character where $\chi_i(g) = d_i$ for some non-identity class, then the group is not simple.

A linear representation is faithful if its kernel is the trivial subgroup. The kernel of a representation $\rho$ρ is the intersection of the kernels of its irreducible constituents. We need to find the smallest representation (possibly reducible) whose kernel is trivial.
*   First, check if the group is simple using the method above.
    *   If the group is simple then every non-trivial irreducible representation is automatically faithful. The answer is the smallest dimension $d_i > 1$ found in the first column of the character table.
    *   If the group is not simple then we must find a set of irreducible characters $\{\chi_{i1}, \chi_{i2}, \dots\}$ such that the intersection of their kernels is trivial. Often, a single irreducible character $\chi_k$ is faithful on its own and we just need to find the one with the smallest dimension.

A faithful permutation representation of smallest dimension corresponds to the action of $G$ on the cosets of a proper subgroup $H$ that has a trivial core, with the smallest possible index $[G:H]$. We wish to find the smallest index or the smallest permutation representation dimension.

*   The table immediately gives us the order of the group $|G| = \sum_i d_i^2$, which may allow us to use the Lagrange's theorem or Sylow's theorems.
*   If G is simple, then the core of any proper subgroup is trivial. Therefore, for a simple group, the smallest dimension of a faithful permutation representation is equal to the smallest possible index of a proper subgroup.
*   While the table does not give the smallest index directly, the dimensions $d_i$ place constraints on the group's structure. For instance, if a group has a subgroup $H$ of index $k$, then $G$ has a permutation character of degree $k$.